In [1]:
# to subpress warning
import warnings
warnings.filterwarnings("ignore")


# The OS module in Python provides functions for interacting with the operating system.
import os

# Matplotlib is a data visualization and graphical plotting library for Python.
import matplotlib.pyplot as plt

# seaborn is alse a data visualization and graphical plotting library for Python.
import seaborn as sn

# Used to display markdown,image,control (frontend utilities)
from IPython.display import display, clear_output

# NumPy is a Python library used for working with arrays
import numpy as np

# used to split dataset(features and target) into test and test
from sklearn.model_selection import train_test_split

# shuffle the dataset for a even mixture of each type of feature and target.it gives better result.
from sklearn.utils import shuffle

# Computer vision library
import cv2

# Used to manipulate different parts of the Python runtime environment.
import sys

# An open-source framework for building pipelines to perform computer vision inference.
import mediapipe as mp

# Time module provide time-related functions
import time

# Used to generate random numbers
import random

from math import log10, sqrt
# for mathematical operations

# Keras is a library that provides a Python interface for artificial neural networks. 
# Keras acts as an interface for the TensorFlow library.
import keras

# 1. Keras layers are the building blocks of the Keras library that can be stacked together for creating neural network models.
# 2. Keras Conv2D creates a 2D convolution kernel that is wind with layers input which helps produce a tensor of outputs.
# 3. maxpooling2D Downsamples the input along its spatial dimensions by taking the maximum value over an input window for each channel of the input. 
# 4 .Flattening is converting the data into a 1-dimensional array for inputting it to the next layer. 
# 5 .Dropout regularization technique for reducing overfitting in neural networks by preventing complex co-adaptations on training data.
# 6. Batch normalization is a technique for training very deep neural networks that standardizes the inputs to a layer for each mini-batch. 
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout, BatchNormalization,Input

#  Model groups layers into an object with training and inference features.
from tensorflow.keras.models import Model

# Optimizers are the extended class, which include added information to train a specific model.  
# The optimizers are used for improving speed and performance for training a specific model.
# Adam is a stochastic gradient descent method based on adaptive estimation of first-order and second-order moments.
from tensorflow.keras.optimizers import Adam

# importing Sqquential model
from tensorflow.keras import Sequential

# tensorflow.keras.callbacks is used to visualize training of a model.
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint 

from tensorflow.keras.applications import VGG16


In [2]:
PATH=r"/kaggle/input/asl-dataset/asl_dataset"

In [3]:
# Loading data from CSV and preprocessing
dataset_path=r"/kaggle/input/d/tirtharajsinha/asl-dataset/dataset.csv"
def load_CSV(dataset):
    # read data from CSV
    rows = open(dataset,"r").readlines()
    totalImage=len(rows)
    
    filenames=[]
    data=[]
    targets=[]
    tempTargets=[]
    start_time = time.time()
    
    # Checking each row
    for i,row in enumerate(rows):
        row=row.replace("\n","")
        row=row.split(",")
        
        visualize(i+1,totalImage,row[0],round(time.time()-start_time,2))
        
        flatPoints=row[3:]
        
        points=[]
        for i in range(0,41,2):
            points.append((int(flatPoints[i]),int(flatPoints[i+1])))
        # Appending the relative file name
        filenames.append(row[0])
        
        targets.append(points)
        tempTargets.append(tuple(points[0]+points[8]))
        
        img=cv2.imread(os.path.join(PATH,row[0]))
        img=cv2.resize(img, (300, 300))
        data.append(img)
        
    return filenames,data,targets,tempTargets

# Progress Visualization block
def visualize(count,total,filename,execTime):
    filename=filename+(40-len(filename))*" "
    clear_output(wait=True)
    prog=(40*count)//total
    percent=(count*100)//total
    line="{}/{} [{}{}] {}%".format(count,total,"#"*prog,"-"*(40-prog),percent)
    print("Analyzing {} | Running for {}s \n{}\r".format(filename,execTime,line))
    
    
    


In [4]:
filenames,data,alltarget,targets=load_CSV(dataset_path)

Analyzing z/hand5_z_dif_seg_5_cropped.jpeg         | Running for 20.21s 
2518/2518 [########################################] 100%


In [5]:
# downsizing and convertion to numpy array
# print(targets)
data = np.array(data, dtype="float32") / 255.0
targets = np.array(targets, dtype="float32")
# print(targets)

In [6]:
# the data for training and the remaining 10% for testing
trainImages,testImages,trainTargets,testTargets,trainFilenames,testFilenames = train_test_split(data, targets, filenames, test_size=0.10,random_state=42)


In [7]:
print(len(trainImages),len(testImages))
print(len(trainTargets), len(testTargets))
print(len(trainFilenames), len(testFilenames))
print(trainImages.shape)
print(trainTargets.shape)

2266 252
2266 252
2266 252
(2266, 300, 300, 3)
(2266, 4)


In [11]:
# KPD23 - Key Point Detection
def kpd23():
    model=Sequential()
    model.add(keras.Input(shape=(300, 300, 3)))
    
    model.add(Conv2D(filters=64, kernel_size=(3,3), padding='same',activation="relu"))
    model.add(Conv2D(filters=64, kernel_size=(3,3), padding='same',activation="relu"))
    model.add(MaxPool2D(pool_size = (2,2), strides = (2,2)))
              
    model.add(Conv2D(filters = 128, kernel_size = (3,3), padding = "same", activation = "relu"))
    model.add(Conv2D(filters = 128, kernel_size = (3,3), padding = "same", activation = "relu"))
    model.add(MaxPool2D(pool_size = (2,2), strides = (2,2)))
    
               
    model.add(Conv2D(filters = 256, kernel_size = (3,3), padding = "same", activation = "relu"))
    model.add(Conv2D(filters = 256, kernel_size = (3,3), padding = "same", activation = "relu"))
    model.add(Conv2D(filters = 256, kernel_size = (3,3), padding = "same", activation = "relu"))
    model.add(MaxPool2D(pool_size = (2,2), strides = (2,2)))
    
    model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = "same", activation = "relu"))
    model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = "same", activation = "relu"))
    model.add(Conv2D(filters = 512, kernel_size = (3,3), padding = "same", activation = "relu"))
    model.add(MaxPool2D(pool_size = (2,2), strides= (2,2)))
                         
    model.add(Conv2D(filters = 1024, kernel_size = (3,3), padding = "same", activation = "relu"))
    model.add(Conv2D(filters = 1024, kernel_size = (3,3), padding = "same", activation = "relu"))
    model.add(Conv2D(filters = 1024, kernel_size = (3,3), padding = "same", activation = "relu"))
    model.add(MaxPool2D(pool_size = (2,2), strides= (2,2)))
    
    model.add(Flatten())
    
    model.add(Dense(units = 1024, activation = "relu"))
    model.add(Dense(units = 512, activation = "relu"))
    model.add(Dense(units = 256, activation = "relu"))
    model.add(Dense(units = 64, activation = "relu"))
    model.add(Dense(units = 4, activation = "sigmoid"))
    
    optimizer = Adam(lr = 0.0001)
    model.compile(optimizer = optimizer, loss="mse")
    
    return model

    
    

In [12]:
model=kpd23()
model.summary()

2022-12-19 10:42:48.048242: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-19 10:42:48.049184: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-19 10:42:48.152669: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-19 10:42:48.153568: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-19 10:42:48.154369: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 300, 300, 64)      1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 300, 300, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 150, 150, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 150, 150, 128)     73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 150, 150, 128)     147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 75, 75, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 75, 75, 256)       2

In [13]:
EPOCH=25
BATCH_SIZE=32

history = model.fit(trainImages, trainTargets,validation_data=(testImages, testTargets),
              batch_size=BATCH_SIZE,epochs=EPOCH,verbose=1)

2022-12-19 10:42:55.832666: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 2447280000 exceeds 10% of free system memory.
2022-12-19 10:42:59.667361: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 2447280000 exceeds 10% of free system memory.
2022-12-19 10:43:01.636128: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/25


2022-12-19 10:43:03.830557: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


71/71 [==============================] - 107s 1s/step - loss: 31521.1660 - val_loss: 31661.9277
Epoch 2/25
71/71 [==============================] - 61s 860ms/step - loss: 31512.1562 - val_loss: 31661.9277
Epoch 3/25
71/71 [==============================] - 62s 874ms/step - loss: 31512.1562 - val_loss: 31661.9277
Epoch 4/25
71/71 [==============================] - 62s 872ms/step - loss: 31512.1562 - val_loss: 31661.9277
Epoch 5/25
71/71 [==============================] - 62s 877ms/step - loss: 31512.1582 - val_loss: 31661.9277
Epoch 6/25
71/71 [==============================] - 62s 875ms/step - loss: 31512.1523 - val_loss: 31661.9277
Epoch 7/25
71/71 [==============================] - 62s 874ms/step - loss: 31512.1445 - val_loss: 31661.9277
Epoch 8/25
71/71 [==============================] - 62s 875ms/step - loss: 31512.1484 - val_loss: 31661.9277
Epoch 9/25
71/71 [==============================] - 62s 875ms/step - loss: 31512.1562 - val_loss: 31661.9277
Epoch 10/25
60/71 [=============

KeyboardInterrupt: 

In [ ]:
model.save("./model.h5", save_format="h5")

In [ ]:
pred = model.predict(testImages)
pred

In [8]:
vgg = VGG16(weights="imagenet", include_top=False,input_tensor=Input(shape=(300, 300, 3)))
# freeze all VGG layers so they will *not* be updated during the
# training process
vgg.trainable = False
# flatten the max-pooling output of VGG
flatten = vgg.output
flatten = Flatten()(flatten)
# construct a fully-connected layer header to output the predicted
# bounding box coordinates
bboxHead = Dense(128, activation="relu")(flatten)
bboxHead = Dense(64, activation="relu")(bboxHead)
bboxHead = Dense(32, activation="relu")(bboxHead)
bboxHead = Dense(4, activation="sigmoid")(bboxHead)
# construct the model we will fine-tune for bounding box regression
model = Model(inputs=vgg.input, outputs=bboxHead)
# initialize the optimizer, compile the model, and show the model
# summary
opt = Adam(lr=0.0001)
model.compile(loss="mse", optimizer=opt)
print(model.summary())
# train the network for bounding box regression
print("[INFO] training bounding box regressor...")
H = model.fit(trainImages, trainTargets,
              validation_data=(testImages, testTargets),
              batch_size=32,
              epochs=25,verbose=1)

2022-12-19 10:55:41.301632: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-19 10:55:41.302673: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-19 10:55:41.408172: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-19 10:55:41.409111: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-19 10:55:41.409881: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 300, 300, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 300, 300, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 300, 300, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 150, 150, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 150, 150, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 150, 150, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 75, 75, 128)       0     

2022-12-19 10:55:45.449150: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 2447280000 exceeds 10% of free system memory.
2022-12-19 10:55:48.311729: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 2447280000 exceeds 10% of free system memory.
2022-12-19 10:55:50.225128: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/25


2022-12-19 10:55:52.199088: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


71/71 [==============================] - 55s 516ms/step - loss: 31515.6816 - val_loss: 31661.9277
Epoch 2/25
71/71 [==============================] - 21s 296ms/step - loss: 31512.1523 - val_loss: 31661.9277
Epoch 3/25
71/71 [==============================] - 22s 305ms/step - loss: 31512.1582 - val_loss: 31661.9277
Epoch 4/25
71/71 [==============================] - 21s 301ms/step - loss: 31512.1484 - val_loss: 31661.9277
Epoch 5/25
71/71 [==============================] - 21s 299ms/step - loss: 31512.1582 - val_loss: 31661.9277
Epoch 6/25
71/71 [==============================] - 21s 299ms/step - loss: 31512.1562 - val_loss: 31661.9277
Epoch 7/25
71/71 [==============================] - 21s 300ms/step - loss: 31512.1523 - val_loss: 31661.9277
Epoch 8/25
71/71 [==============================] - 21s 302ms/step - loss: 31512.1484 - val_loss: 31661.9277
Epoch 9/25
71/71 [==============================] - 21s 301ms/step - loss: 31512.1484 - val_loss: 31661.9277
Epoch 10/25
71/71 [===========

In [9]:
print("[INFO] saving object detector model...")
model.save("./model_vgg.h5", save_format="h5")

[INFO] saving object detector model...


In [10]:
pred = model.predict(testImages)
pred

array([[1.       , 1.       , 1.       , 1.       ],
       [1.       , 1.       , 1.       , 1.       ],
       [1.       , 1.       , 1.       , 1.       ],
       ...,
       [1.       , 1.       , 1.       , 1.       ],
       [1.       , 1.       , 1.       , 1.       ],
       [1.       , 1.       , 0.9999999, 1.       ]], dtype=float32)

In [ ]:
testTargets